In [ ]:

!pip install gTTS SpeechRecognition wikipedia-api pydub

In [ ]:
# Célula 2: Código para Gravação e Transcrição de Áudio

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io

import speech_recognition as sr

# --- Snippet de JavaScript para gravação de áudio ---
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Pressione para Falar");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Gravando... Pressione para Parar.";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Salvando gravação..."
  }
}

function dataURItoBlob(dataURI) {
  var byteString = atob(dataURI.split(',')[1]);
  var mimeString = dataURI.split(',')[0].split(':')[1].split(';')[0]
  var ab = new ArrayBuffer(byteString.length);
  var ia = new Uint8Array(ab);
  for (var i = 0; i < byteString.length; i++) {
      ia[i] = byteString.charCodeAt(i);
  }
  return new Blob([ab], {type: mimeString});
}

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("base64data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]

  sr_audio = io.BytesIO(riff)
  with sr.AudioFile(sr_audio) as source:
    audio = sr.Recognizer().record(source)

  return audio

def speech_to_text():
    """Grava o áudio do usuário e retorna o texto transcrito."""
    print("Ouvindo...")
    audio = get_audio()
    recognizer = sr.Recognizer()
    try:
        texto = recognizer.recognize_google(audio, language='pt-BR')
        print(f"Você disse: {texto}")
        return texto.lower()
    except sr.UnknownValueError:
        print("Não entendi o que você disse.")
        return ""
    except sr.RequestError as e:
        print(f"Erro no serviço de reconhecimento de fala; {e}")
        return ""

In [ ]:

import wikipediaapi
import webbrowser


wiki_wiki = wikipediaapi.Wikipedia('VirtualAssistantProject/1.0', 'pt')

def pesquisar_wikipedia(termo):
    """Pesquisa um termo na Wikipedia e retorna um resumo."""
    print(f"Pesquisando '{termo}' na Wikipedia...")
    pagina = wiki_wiki.page(termo)
    if not pagina.exists():
        return f"Não encontrei a página para '{termo}' na Wikipedia."
    return f"Segundo a Wikipedia, {pagina.summary.split('.')[0]}."

def abrir_youtube(pesquisa):
    """Gera um link de busca para o YouTube."""
    # No Colab, não podemos abrir uma aba diretamente, então geramos o link
    termo_busca = pesquisa.replace(' ', '+')
    url = f"https://www.youtube.com/results?search_query={termo_busca}"
    print(f"Para abrir o YouTube, clique no link: {url}")
    return f"Aqui está um link para sua busca sobre '{pesquisa}' no YouTube."

def encontrar_farmacia():
    """Gera um link do Google Maps para farmácias próximas."""
    # Também geramos um link, pois o Colab não tem acesso à sua localização.
    url = "https://www.google.com/maps/search/farmácia"
    print(f"Para encontrar farmácias próximas, clique no link: {url}")
    return "Mostrando farmácias próximas no Google Maps."

In [ ]:
# Célula 4: Módulo Text-to-Speech
from gtts import gTTS
from IPython.display import Audio, display

def text_to_speech(texto, lang='pt'):
    """Converte texto em áudio e o reproduz no Colab."""
    try:
        tts = gTTS(text=texto, lang=lang)
        # Salva o áudio em um arquivo temporário em memória
        audio_fp = io.BytesIO()
        tts.write_to_fp(audio_fp)
        audio_fp.seek(0) # Volta para o início do arquivo em memória
        display(Audio(audio_fp.read(), autoplay=True))
    except Exception as e:
        print(f"Ocorreu um erro no Text-to-Speech: {e}")

In [ ]:

def processar_comando(comando):
    """Analisa o comando de voz e executa a ação correspondente."""
    resposta = "Desculpe, não entendi o comando."

    if 'wikipédia' in comando:
        # Extrai o termo de busca (o que vem depois de 'wikipédia')
        termo = comando.split('wikipédia', 1)[-1].strip()
        if termo:
            resposta = pesquisar_wikipedia(termo)
        else:
            resposta = "Por favor, diga o que você quer pesquisar na Wikipedia."

    elif 'youtube' in comando:
        termo = comando.split('youtube', 1)[-1].strip()
        if termo:
            resposta = abrir_youtube(termo)
        else:
            resposta = "Por favor, diga o que você quer pesquisar no YouTube."

    elif 'farmácia' in comando or 'farmácias' in comando:
        resposta = encontrar_farmacia()

    return resposta

In [ ]:

print("Olá! Sou seu assistente virtual. Diga um comando.")
text_to_speech("Olá! Sou seu assistente virtual. Diga um comando.")


comando_de_voz = speech_to_text()

if comando_de_voz:

    resposta_texto = processar_comando(comando_de_voz)

    print(f"Assistente: {resposta_texto}")
    text_to_speech(resposta_texto)